In [1]:
include("src\\ImportanceSampling.jl")
using ImportanceSampling
using Distributions

In [2]:
# defined by θ (shape), not λ (rate)
struct MvExponential <: ContinuousMultivariateDistribution
    Es::Vector{Exponential{Float64}}

    function MvExponential(θs::Vector{<:Real})
        all(θs .> 0) || error("all θs must be > 0")
    
        return new(Exponential.(float(θs)))
    end
end

MvExponential(n::Integer) = MvExponential(ones(n))
Base.length(d::MvExponential) = length(d.Es)
Distributions._rand!(d::MvExponential, x::AbstractVector{<:Real}) = 
    x .= rand.(d.Es)
Distributions._logpdf(d::MvExponential, x::AbstractVector{<:Real}) = 
    sum(logpdf(d.Es[i], x[i]) for i in 1:length(d))

In [15]:
function f!(p, x)
    E1 = x[1]
    E2 = E1 + x[2]
    E3 = E1 + x[3]
    E4 = E2 + x[4]
    E5 = E2 + x[5]
    E6 = E3 + x[6]
    E7 = E3 + x[7]
    E8 = E3 + x[8]
    E9 = max(E5, E6, E7) + x[9]
    S10 = max(E4, E8, E9)
    p[1] = min(1, exp(-(70-S10)/θs[10]))
end

f! (generic function with 1 method)

In [16]:
θs = [4, 4, 2, 5, 2, 3, 2, 3, 2, 2]
critpath = zeros(10)
critpath[[1, 2, 4, 10]] = 1
p = MvExponential(θs)
q = MixtureDistribution([p, MvExponential((1+2*critpath).*θs), 
        MvExponential((1+4*critpath).*θs)], [1, 8, 1])

is = ImportanceSampler(f!, 1, q, p=p)
;

In [17]:
update!(is, niters=10_000, nbatches=10)
;

In [ ]:
mean(is)

In [ ]:
sqrt(var(is))

In [ ]:
ne(is)

In [ ]:
neσ(is)

In [18]:
neγ(is)

(3.3882994458725415e-5, 1.8826937629001455e-6, 3668.573204040512, 1464.1844980738745, 2022.2120991057361)